In [ ]:
import pandas as pd

import numpy as np
from tqdm import tqdm

from numba import jit

import matplotlib.pyplot as plt
#import seaborn as sns

import matplotlib.pyplot as plt

In [ ]:
# read in the dataframe you want to clean, can be a csv, hdf
#traindf = pd.read_csv("./data/training_set_VU_DM.csv")
traindf = pd.read_pickle("./pickles/df_small_train.pkl")

In [ ]:
# histogram of differences in prices between searches with or without saturdaynight
with_saturday = traindf.loc[traindf['srch_saturday_night_bool'] == 1]['price_usd'].to_numpy()

without_saturday = traindf.loc[traindf['srch_saturday_night_bool'] == 0]['price_usd'].to_numpy()



In [ ]:
# traindf["price_usd"][traindf["srch_saturday_night_bool"] == 1 & (traindf['price_usd'] < 1000)].hist(bins=10000)

In [ ]:
plt.hist([with_saturday, without_saturday], alpha=0.5, bins=10000, label=['with saturday', 'without saturday'])
plt.show()

In [ ]:
@jit(nopython=True, parallel=True)
def calculate_mean_per_propid(unique_propids, 
                                    col_propids, 
                                    col_price, 
                                    col_mean, 
                                    col_std, 
                                    col_no_hotels, 
                                    col_correct_price, 
                                    col_nights,
                                    col_new_mean,
                                    col_new_std):
    
    total = len(unique_propids)

    for i, propid in enumerate(unique_propids):
        
        if i % 1000 == 0:
            print(i, "/", total)
        
        indices = np.where(col_propids == propid)[0]

        prices = col_price[indices]
        mean_price = np.mean(prices)
        std_price = np.std(prices)
        
        col_mean[indices] = mean_price
        col_std[indices] = std_price
        col_no_hotels[indices] = indices.size
        
        if std_price > 100:
            col_correct_price[indices] = col_price[indices] / col_nights[indices]
            prices = col_correct_price[indices]
            mean_price = np.mean(prices)
            std_price = np.std(prices)
            
        col_new_mean[indices] = mean_price
        col_new_std[indices] = std_price

        return col_mean, col_no_hotels

In [ ]:
def compute_numba(df):
    # add columns
    columnames = list(df.columns)
    columnames.extend(["avg_price_propid", "std_avg_price_propid", "amount_hotels", 
                       "avg_price_propid_after", "std_avg_price_propid_after"])

    df = df.reindex(columns=columnames)

    # copy prices (for now, at the end we will just update the price i suppose)
    df["price_correction"] = df["price_usd"]

    # get unique propids and put them into numpy array
    propids = np.array(df["prop_id"].unique())
   
    means, no_hotels = calculate_mean_per_propid(propids,
                                        df['prop_id'].to_numpy(),
                                        df['price_usd'].to_numpy(),
                                        df['avg_price_propid'].to_numpy(),
                                        df['std_avg_price_propid'].to_numpy(),
                                        df['amount_hotels'].to_numpy(),
                                        df['price_correction'].to_numpy(),
                                        df['srch_length_of_stay'].to_numpy(),
                                        df['avg_price_propid_after'].to_numpy(),
                                        df['std_avg_price_propid_after'].to_numpy())    
    return df

In [ ]:
print("TEST")
df = compute_numba(traindf)
display(df[['prop_id', 'amount_hotels', 'avg_price_propid', 'price_usd', 'std_avg_price_propid', 'srch_length_of_stay', 'price_correction', 'avg_price_propid_after', 'std_avg_price_propid_after']][df['prop_id'] == 37304])
display(df[['prop_id', 'amount_hotels', 'avg_price_propid', 'price_usd', 'std_avg_price_propid', 'srch_length_of_stay', 'price_correction', 'avg_price_propid_after', 'std_avg_price_propid_after']])
print(len(df['amount_hotels'].notna()))

In [ ]:
arr = np.array([11, 12, 13, 14, 15, 16, 17, 15, 11, 12, 14, 15, 16, 17])
indices = np.where(arr==15)
print(indices)
print(indices[0].size)

In [ ]:
print("Std before: ", df['std_avg_price_propid'].mean())
print("Std after: ", df['std_avg_price_propid_after'].mean())
df['std_avg_price_propid'][df['std_avg_price_propid'] < 2000].hist(bins=100)

plt.figure()
df['std_avg_price_propid_after'][df['std_avg_price_propid'] < 2000].hist(bins=100)

In [ ]:
display(df[['prop_id','std_avg_price_propid', 'std_avg_price_propid_after']][df['std_avg_price_propid_after'] > 10000])
display(df[df['prop_id'] == 13878])

display(df[['prop_id', 'amount_hotels', 'avg_price_propid', 'price_usd', 'std_avg_price_propid', 'srch_length_of_stay', 'price_correction', 'avg_price_propid_after', 'std_avg_price_propid_after']][df['prop_id'] == 13878])

In [ ]:
df.to_hdf("corrected_price.hdf", key='df')